In [4]:
import pandas as pd
import requests

In [5]:
res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('ratings_train.txt', 'wb') as f:
    f.write(res.content)

nsmc = pd.read_csv('ratings_train.txt', sep='\t')
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 전처리

In [6]:
import re    # 정규 표현식

In [7]:
# 한글만 찾아주는 함수
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [8]:
nsmc.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [9]:
nsmc['document'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
149995    True
149996    True
149997    True
149998    True
149999    True
Name: document, Length: 150000, dtype: bool

In [10]:
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [14]:
# 비어있는 행을 제외한 데이터에서 한글만 추출
data = nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [15]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [16]:
data[1]

['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']

In [17]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [18]:
only_hangul(nsmc.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [19]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [20]:
data2[0]

'아 더빙 진짜 짜증나네요 목소리'

In [21]:
with open('nsmc.txt', 'w', encoding="utf8") as f:
    f.write('\n'.join(data2))

## FastText 모형 학습

In [22]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [24]:
# sb = 0(기본값) : (cbow학습), s = 1 : skip-gram학습
# alpha = 0.025 : 학습률
# min_alpha : 학습률을 낮춰주는 역할
# window = 5 : 좌우 몇개 단어 맥락으로 학습을 할지 결정
# min_count = 5 : 최소 몇번 나와야 학습을 함
# vector_size = 100(기본값)
model = FastText(vector_size=16)

In [25]:
model.build_vocab(corpus_iterable=data)

In [27]:
model.train(corpus_iterable=data,
            epochs=5,
            total_examples=model.corpus_count,
            total_words=model.corpus_total_words
)

(3999341, 5829395)

## 저장

In [28]:
model.save('nsmc.fasttext')

## 임베딩

In [29]:
model = FastText.load('nsmc.fasttext')

In [30]:
model.wv.key_to_index

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [31]:
'히어로' in model.wv.key_to_index   # '히어로' in model.wv.vocab : gensim 4.0 미만

True

In [32]:
model.wv['히어로']

array([-0.37215963,  0.64510006,  0.48477873,  0.3916511 ,  0.8176526 ,
        0.25969025, -0.5894319 , -0.50926787,  0.25958887, -0.04006614,
        0.2666501 , -0.9485264 ,  0.20394005, -0.32159343, -0.05420427,
        0.3871244 ], dtype=float32)

In [33]:
len(model.wv['히어로'])

16

In [34]:
'슈퍼히어로' in model.wv.key_to_index

False

In [35]:
model.wv['슈퍼히어로']

array([-0.16530372,  0.25838432,  0.20821239,  0.16901389,  0.3265664 ,
        0.11895531, -0.21204224, -0.21386346,  0.14158247, -0.02526328,
        0.1401457 , -0.36346593,  0.04522087, -0.11869159,  0.00428683,
        0.17396341], dtype=float32)

## 유사도

In [36]:
model.wv.similarity('슈퍼히어로','히어로')  # 코사인 유사도로 비교

0.99425983

In [37]:
model.wv.similarity('히어로','평론가')

0.73220867

In [38]:
model.wv.most_similar('평론가')

[('평론가들', 0.9917201995849609),
 ('평론', 0.9916314482688904),
 ('점대면', 0.9908243417739868),
 ('점대나', 0.9900078773498535),
 ('점대지', 0.9893202185630798),
 ('점대가', 0.9892838597297668),
 ('점대야', 0.9877073764801025),
 ('점대냐', 0.9870886206626892),
 ('점대라', 0.987014651298523),
 ('점이야', 0.986345648765564)]

## FastText를 이용한 감성분석

In [39]:
from gensim.models.fasttext import FastText

ft = FastText.load('nsmc.fasttext')

In [41]:
import pandas as pd

nsmc = pd.read_csv('ratings_train.txt', sep='\t')

In [42]:
df = nsmc[nsmc['document'].notnull()]

In [45]:
from sklearn.model_selection import train_test_split

In [48]:
doc_train, doc_test, y_train, y_test = train_test_split(df['document'], df['label'], \
                                    test_size=0.2, random_state=42)

In [49]:
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+', text)

In [50]:
import numpy as np
x_train = np.zeros((1000, 16))

In [52]:
doc_train.shape

(119996,)

In [53]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [54]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    print(find_hangul(doc))

['아', '꿀잼ㅋ', '친구랑', '봤는데', '너무', '웃겼음', '그리구', '김우빈', '잘생겼다']
['개건의', '졸작', '스릴러', '스릴러라고', '하게에도', '민망하군']
['장하나', '한윤찬', '제발', '이어주세요', '말도안되게', '왜', '설도현과']
['애로영화계의', '개']
['내용이나', '그래픽자체가', '허접함']
['요새', '이런', '영화가', '드물다는게', '안타깝다']
['역사적', '의의를', '떠나서', '재밌습니다', '지금봐도', '손색이', '없습니다']
['핵꿀잼']
['책을', '읽은', '느낌', '사랑의', '을', '보여줬다']
['평점', '말도안돼']
['난', '역시', '천재야', '개월', '전에', '본', '영화', '기억해냈당']
['재밌음']
['살', '조카는', '무한점', '저는', '점', '가끔', '웃기기도', '하고', '단', '안경', '좀', '불편', '약간', '어지러움']
['하아', '암걸릴뻔']
['태어나서', '별점처음', '써본다', '이런게', '스릴러지', '유해진', '진구씨', '연기쩔어', '왜이제', '봤을까', '너무재밌음']
['나의', '계왕원백배', '원기옥', '기원참', '에네르기파', '마광관살포를', '받아랏']
['많은것을', '말하려', '했지만', '한마디도', '하지', '못한', '영화']
['옴니버스', '호러물로', '한국', '호러물중', '최고의', '영화가', '아닐까']
['내', '인생', '최고의', '강시영화']
['아오걍하지마', '뭔고학년도안된아이를가지고장난이야걍민국이하고지아는', '아빠어디가에서만하지', '왜영화까지끌고나와서', '더이미지안좋게하는건데']
['굿']
['년에', '보고', '년에', '다시', '봐도', '역시나', '최고의', '드라마다']
['연출', '내용', '연기', '음악', '조명', '스타일링', '전부', '굿', '그리고', '이연희의', '재발견']
[

In [66]:
for i, doc in enumerate(doc_train.iloc[:1000]):
    # for word in find_hangul(doc):
    #     print(ft.wv[word])
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    if vs:
        # print(doc)
        # print(len(vs))
        # print(vs[0])
        x_train[i,] = np.mean(vs, axis=0)
    else:
        print(doc)

once upon a dream
good
good
it's jjangge movie
Good message
